# tools for planning observations

In [ ]:
__author__ = 'Pascal Louis <plouis35@gmail.com>'
__version__ = '0.1 alpha'
__keywords__ = ['astronomy', 'astropy', 'astroquery', 'matplotlib']


## Configuration

In [ ]:
# CALC
obs_latitude = 47.89
obs_longitude = -1.50

target = 'RR lyr'
obs_time = '2023-11-07'      # local time


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from astropy.visualization import astropy_mpl_style, quantity_support

plt.style.use(astropy_mpl_style)
quantity_support()


## Show alt and moon/sun rise and set

In [ ]:
from astropy.coordinates import get_sun
from astropy.coordinates import get_body

### get SUN position
delta_midnight = np.linspace(-12, 12, 1000)*u.hour
times_to_tomorow = midnight + delta_midnight
frame_to_tomorow = AltAz(obstime=times_to_tomorow, location=obs_loc)
sunaltazs_to_tomorow = get_sun(times_to_tomorow).transform_to(frame_to_tomorow)

### get MOON position
moon_to_tomorow = get_body("moon", times_to_tomorow)
moonaltazs_to_tomorow= moon_to_tomorow.transform_to(frame_to_tomorow)
obj_altazs_to_tomorow= obj.transform_to(frame_to_tomorow)

plt.plot(delta_midnight, sunaltazs_to_tomorow.alt, color='r', label='Sun')
plt.plot(delta_midnight, moonaltazs_to_tomorow.alt, color=[0.75]*3, ls='--', label='Moon')
plt.scatter(delta_midnight, obj_altazs_to_tomorow.alt,
            c=obj_altazs_to_tomorow.az.value, label=target, lw=0, s=8,
            cmap='viridis')
plt.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 sunaltazs_to_tomorow.alt < -0*u.deg, color='0.5', zorder=0)
plt.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 sunaltazs_to_tomorow.alt < -18*u.deg, color='k', zorder=0)
plt.colorbar().set_label('Azimuth [deg]')
plt.legend(loc='upper left')
plt.xlim(-12*u.hour, 12*u.hour)
plt.xticks((np.arange(13)*2-12)*u.hour)
plt.ylim(0*u.deg, 90*u.deg)
plt.xlabel('Hours from EDT Midnight')
plt.ylabel('Altitude [deg]')
plt.show()


## Show target details

In [ ]:
#import module
from astroquery.simbad import Simbad

#get a simbad instance
simbad = Simbad()

#add spectral type parameters for VOTable request
simbad.add_votable_fields('sptype')

#request
result_table = simbad.query_object(target)
#result_table.pprint(show_unit=True)
print(repr(result_table))

#Coordinates
print("\nCoordinates")
print(result_table['RA'][0])
print(result_table['DEC'][0])

#Spectral Type
print("\nSpectral Type")
print(result_table['SP_TYPE'])

## Show field of view

In [ ]:
from astroplan.plots import plot_finder_image
from astroplan import FixedTarget
import matplotlib.pyplot as plt
from astroquery.skyview import SkyView

obj_ = FixedTarget.from_name(target)
ax, hdu = plot_finder_image(obj_, survey = "DSS", reticle = False, fov_radius=6*u.arcminute)
plt.show()

#'Optical:SDSS': ['SDSSg',

In [ ]:
#from astroquery.skyview import SkyView; SkyView.list_surveys()

In [ ]:
from ipyaladin import Aladin
aladin = Aladin(fov=0.1, target = 'm33', height = 600)
aladin